### Create spark session

In [1]:
cd .. 

/home/guava/projects/learn-spark


In [2]:
from src.utils import create_spark_session

In [3]:
spark = create_spark_session('Structured Operations')

# Basic Structured Operations

Recap:

- DataFrame: consist of records (**Row**) AND columns
- Schema: define name and type of columns
- Partition: physical distribution of the data across cluster
- Partitioning scheme: how that is allocated (i.e. based on values in a certain column or nondeterministically.)

In [16]:
df = spark.read.format("json")\
    .load('data/flight-data/json/2015-summary.json')


In [6]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



### Schema best practices

|   | Manually set the schema | Schema-on-read (schema inference) |
| --- | --- | -- |
| Scenario | Production | Ad-hoc |
| Speed | Faster | slower on plain-text file e.g. CSV /JSON |
| Data Type | Safe | schema inference is not stable on untyped data i.e. long type may be interpreted as integer |

- *schema* = `StructType` consist of a number of 
- *fields* = `StructFields(name, type, nullable: bool)`
- Also, *metadata* can be used to associate with a column

At *runtime*, if types in the data do not match with the schema, an *error* will be thrown

In [9]:
# schema on read
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [17]:
# Manually set up the schema
from pyspark.sql.types import StructField, StructType, StringType, LongType


myManualSchema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
    StructField("count", LongType(), False, metadata={"hello": "world"})
])
df = spark.read.format("json").schema(myManualSchema)\
    .load("data/flight-data/json/2015-summary.json")

## Columns and Expressions

To Spark, columns are logical constructions that simply represent a value computed on a perrecord basis by means of an expression.

### Columns

Refer to columns, two simplest ways are 

- `col` & `column`

Note: column might or might not exist in DataFrames. They are not *resolved* until they are compared with those in the *catalog*. The resolution happens in the *analyzer* phase.

In [25]:
from pyspark.sql.functions import col, column

col("TestCol")
column("TestCol")

Column<b'TestCol'>

#### Explicit column reference

- `.col` method on a dataframe
- Spark will not resolve this column in the *analyzer* phase, because we did it

In [26]:
try:
    # invalid column
    df.col('invalid')
except:
    print('This is an invalid column. An error will be raised right away')

This is an invalid column. An error will be raised right away


### Expressions

An expression is a set of transformations on one or more values in a record in a DataFrame.

`expr("someCol")` == `col("someCol")`

#### Columns as expressions

- When use `col()` for transformation, you need to do it on the column reference i.e. `col("someCol") - 5`
- However, with `expr()`, transformations and column references can go together i.e. `expr("someCol - 5")`

> Because Spark parse compile these into a logical tree with the order of operations. Key points:

- Columns are just expressions
- Columns and transformation of those columns compile to the same logical plan as parsed expressions.

In [27]:
from pyspark.sql.functions import expr

expr("(((someCol + 5) * 200) - 6) < otherCol")

Column<b'((((someCol + 5) * 200) - 6) < otherCol)'>

> The logical tree. Both DataFrame & SQL code will compile to this logical tree and enjoy the same performance characteristic

![](img/logical-tree.jpg)

#### Accessing a DataFrame’s columns

In [29]:
# to list all the columns 
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

### Records and Rows

In [30]:
# list the first row
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

#### Creating Rows

- *only* DataFrames have schema
- `Row` does not schema

In [31]:
from pyspark.sql import Row

myRow = Row("Hello", None, 1, False)

In [33]:
# note: in Scala or Java, you must need to explicitly coerce the value or using helper methods
myRow[0]

'Hello'

In [34]:
myRow[2]

1

### DataFrame Transformations

![](img/df-transformations.jpg)

#### Create DataFrames

In [36]:
# create a SQL view
df.createOrReplaceTempView("dfTable")

In [49]:
# manually create data frame by with Rows
myManualSchema = StructType([
    StructField("some", StringType(), True),
    StructField("col", StringType(), True),
    StructField("names", LongType(), False)
])
myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow, myRow] * 3, myManualSchema)
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
|Hello|null|    1|
|Hello|null|    1|
|Hello|null|    1|
|Hello|null|    1|
|Hello|null|    1|
+-----+----+-----+



#### `select` and `selectExpr`

they are similar to SQL

In [53]:
# SELECT DEST_COUNTRY_NAME FROM dfTable LIMIT 2
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [54]:
# SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME FROM dfTable LIMIT 2
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [59]:
# expr == col == column
df.select(
    expr("DEST_COUNTRY_NAME"),
    col("DEST_COUNTRY_NAME"),
    column("DEST_COUNTRY_NAME"))\
.show(2)

# Book: `Column` object now does not like `string`
# Update: Now they actually like each other

df.select(col("DEST_COUNTRY_NAME"), "DEST_COUNTRY_NAME")


+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



DataFrame[DEST_COUNTRY_NAME: string, DEST_COUNTRY_NAME: string]

In [67]:
# expr is the most flexible 
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

# use `AS` then `.alias` to change it back
df.select(expr("DEST_COUNTRY_NAME AS destination").alias("DEST_COUNTRY_NAME"))\
    .show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [68]:
# `select` followed by `expr` == selectExpr
df.selectExpr("DEST_COUNTRY_NAME AS newColumn", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|    newColumn|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [69]:
df.selectExpr(
    "*",
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry"
).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [70]:
# aggregations
df.selectExpr(
    "AVG(count)",
    "COUNT(DISTINCT(DEST_COUNTRY_NAME))"
).show(2)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



### Converting to Spark Types (Literals)

In [73]:
# literal ~ broadcasting
from pyspark.sql.functions import lit

# SELECT *, 1 as One FROM dfTable LIMIT 2
df.select(
    '*',
    lit(1).alias('One')
).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



### Adding Columns

In [76]:
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [77]:
df.withColumn(
    "withinCountry", 
    expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")
).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [83]:
# add a new new column that is identical to another column
df.withColumn("Destination", expr("DEST_COUNTRY_NAME")).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

### Renaming Columns

In [81]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

### Reserved Characters and Keywords

- use backtick `

In [84]:
# you don't need to escape here
# it is not a `expr` but just a string 
dfWithLongColName = df.withColumn(
    "This Long Column-Name",
    expr("ORIGIN_COUNTRY_NAME")
)

In [89]:
# here we need ` because we are using `expr` here
dfWithLongColName.selectExpr(
    "`This Long Column-Name`",
    "`This Long Column-Name` AS `new col`"
).show(2)

# equivalent to 
# SELECT `This Long Column-Name`, `This Long Column-Name` as `new col`
# FROM dfTableLong LIMIT 2

+---------------------+-------+
|This Long Column-Name|new col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows



Note: you only need \` in `expr` but not in `string-to-column` reference

In [93]:
# this is valid
dfWithLongColName.select(col("This Long Column-Name")).columns

# this is not 
try:
    dfWithLongColName.select(expr("This Long Column-Name")).columns
except:
    print('Need to do this instead: expr("`This Long Column-Name`")')

Need to do this instead: expr("`This Long Column-Name`")


### Case Sensitivity

- by default: Spark is *case insensitive*

you can change it by 

```sql
-- in SQL
set spark.sql.caseSensitive true
```

### Removing Columns

In [94]:
df.drop("ORIGIN_COUNTRY_NAME").columns

['DEST_COUNTRY_NAME', 'count']

In [95]:
dfWithLongColName.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME")

DataFrame[count: bigint, This Long Column-Name: string]

### Column type casting

example

```sql
-- in SQL
SELECT *, cast(count as long) AS count2 FROM dfTable
```

In [96]:
df.withColumn("count2", col("count").cast("long"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: bigint]

### Filtering Rows

- `.filter` & `.where`

In [97]:
df.filter(col("count") < 2).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [100]:
df.where("count < 2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [103]:
df\
    .filter(col("count") < 2)\
    .filter(col("ORIGIN_COUNTRY_NAME") != "Croatia")\
    .show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



### Unique rows

`.distinct`

```sql
-- in SQL
SELECT COUNT(DISTINCT(ORIGIN_COUNTRY_NAME, DEST_COUNTRY_NAME)) FROM dfTable
```

In [105]:
df.select('ORIGIN_COUNTRY_NAME', 'DEST_COUNTRY_NAME')\
    .distinct()\
    .count()

256

In [106]:
df.select('ORIGIN_COUNTRY_NAME')\
    .distinct()\
    .count()

125

### Random samples

In [107]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

126